In [ ]:
import os
import zipfile

local_zip = 'wayang-new.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('.')
zip_ref.close()

In [32]:
base_dir = './new/'

wayang_classes = os.listdir(base_dir)
print(wayang_classes)

['bagong', 'gareng', 'petruk', 'semar']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
image_batch_size = 20
aug_image_total = 60

new_datagen = ImageDataGenerator(rescale = 1.0/255,
                                 horizontal_flip = True,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 fill_mode = 'nearest',
                                 rotation_range=15,
                                 zoom_range=0.2)

In [41]:
for wayang_class in wayang_classes:
    aug_image_step = 0
    new_generator = new_datagen.flow_from_directory(base_dir, batch_size=image_batch_size,
                                                    class_mode='binary', classes=[wayang_class],
                                                    target_size=(225, 175),
                                                    shuffle=False, save_to_dir='/'.join(['augmented', wayang_class]),
                                                    save_prefix='', save_format='jpg')
    for image_batch in new_generator:
        aug_image_step += 1
        if aug_image_step > aug_image_total:
            break

Found 85 images belonging to 1 classes.
Found 55 images belonging to 1 classes.
Found 72 images belonging to 1 classes.
Found 99 images belonging to 1 classes.


In [44]:
augmented_dir = './augmented/'

train_batch = 20
validation_batch = 20

In [46]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                 horizontal_flip = True,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 fill_mode = 'nearest',
                                 rotation_range=15,
                                 zoom_range=0.2,
                                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(augmented_dir, batch_size=train_batch,
                                                    class_mode='categorical', target_size=(225, 175),
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(augmented_dir, batch_size=validation_batch,
                                                         class_mode='categorical', target_size=(225, 175),
                                                         subset='validation')

Found 3575 images belonging to 4 classes.
Found 893 images belonging to 4 classes.
